# Explore here

In [6]:
from flask import Flask, request, render_template, send_file
from transformers import AutoModelForImageSegmentation
import torch
from torchvision import transforms
from PIL import Image
import io

In [7]:
app = Flask(__name__)

In [8]:
# Detectar si hay GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
# Cargar el modelo solo una vez
model = AutoModelForImageSegmentation.from_pretrained(
    "ZhengPeng7/BiRefNet", trust_remote_code=True
).to(device)

In [10]:
# Transformaciones
transform_image = transforms.Compose([
    transforms.Resize((1024, 1024)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [11]:
@app.route("/", methods=["GET", "POST"])
def index():
    if request.method == "POST":
        file = request.files["image"]
        image = Image.open(file.stream).convert("RGB")
        processed_image = remove_background(image)
        
        # Convertimos la imagen a bytes
        img_io = io.BytesIO()
        processed_image.save(img_io, format="PNG")
        img_io.seek(0)
        
        return send_file(img_io, mimetype='image/png')
    
    return '''
        <h1>Background Remover</h1>
        <form method="POST" enctype="multipart/form-data">
            <input type="file" name="image">
            <input type="submit" value="Process">
        </form>
    '''

In [12]:
def remove_background(image):
    original_size = image.size
    input_tensor = transform_image(image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        preds = model(input_tensor)[-1].sigmoid().cpu()
    
    pred = preds[0].squeeze()
    pred_pil = transforms.ToPILImage()(pred)
    mask = pred_pil.resize(original_size)
    
    image.putalpha(mask)
    return image

In [ ]:
if __name__ == "__main__":
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/workspaces/ML_WEB_APP_USING_FLASK_JesusCastanedam/.venv/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/workspaces/ML_WEB_APP_USING_FLASK_JesusCastanedam/.venv/lib/python3.11/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/workspaces/ML_WEB_APP_USING_FLASK_JesusCastanedam/.venv/lib/python3.11/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/ML_WEB_APP_USING_FLASK_JesusCastanedam/.venv/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 692, in initialize
    self.init_sockets()
  File "/workspaces/ML_WEB_APP_US

SystemExit: 1

/workspaces/ML_WEB_APP_USING_FLASK_JesusCastanedam/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3678: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


: 